In [1]:
import fasttext
import re
import numpy as np

## Dataset
dataset = [
    "မြန်မာ ပြည် ရှေ့ရေး အကြောင်း ဆွေးနွေးသွားတာ အရမ်း ကောင်းတယ်။", 
    "အရမ်း မှန်တဲ့ စကားတွေပါ။ ကျေးဇူး တင်ပါတယ်။", 
    "တန်ဖိုး ရှိတဲ့ စကားတွေပါ။ လေးစားပါတယ်။", 
    "သုံးသပ်သွားတာ ကောင်းလိုက်တာ။ အရမ်း ဗဟုသုတ ရတယ်။", 
    "ကျေးဇူးပါ ဆရာ သုံးသပ်ချက် တွေကို ကျေနပ် အားရပါတယ်။", 

    "မင်းဆိုး မင်းညစ် အယုတ်တမာ ကျဆုံးမှာပါ။", 
    "ကလိမ်ကကျစ် နဲ့ စကား ကို ဝေ့ဝိုက်ပြီး ပြောနေတယ်။", 
    "မိုက်ရိုက်လိုက်တဲ့ လုပ်ရပ်ဗျာ။ အတော် ဆိုးတာပဲ။", 
    "ကျက်သရေ တုံးပါကွာ။ အရည်အချင်းကို မရှိဘူး။", 
    "မင်းကို မြင်ရတာ စိတ် ကသိကအောက် ဖြစ်တယ်။", 

    "ပြည်ထောင်စု သမ္မတ မြန်မာ နိုင်ငံတော် ထာဝရ တည်တံ့နိုင်ပါစေ။", 
    "ပြည်ထောင်စု ဖွား တိုင်းရင်းသားတွေ စုစည်းနေရင် အရမ်း ကောင်းမှာပဲ။", 
    "စစ်ပွဲတွေ ကို မုန်းလိုက်တာ ဗျာ။", 
    "စာသင်ကျောင်း တံခါးများ အမြဲတမ်း ဖွင့်ထားနိုင်ပါစေ။", 
    "ရခိုင် မုန့်တီ၊ ရှမ်း ခေါက်ဆွဲ နဲ့ ကရင် ကုတိယို ကို အရမ်း ကြိုက်တယ်။" 
]
targets = ["Positive", "Positive", "Positive", "Positive", "Positive", "Negative", "Negative", "Negative", "Negative", "Negative", "Neutral", "Neutral", "Neutral", "Neutral", "Neutral"]

## Convert the target labels to numerical values
target_mapping = {"Positive": 0, "Negative": 1, "Neutral": 2}
encoded_lbls = np.array([target_mapping[target] for target in targets])
print(f'Encoded Label : {encoded_lbls}')

Encoded Label : [0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


In [6]:
## Preprocess dataset: Tokenization and cleaning
def preprocess_text(text):
    ## Lowercase the text
    text = text.lower()
    ## Remove punctuation and special characters
    # text = text.split() 
    return text

## Prepare FastText compatible training data by combining sentence and label
with open("fasttext_train.txt", "w") as f:
    for i, sentence in enumerate(dataset):
        cleaned_sentence = preprocess_text(sentence)
        label = targets[i]
        f.write(f"__label__{label} {cleaned_sentence}\n")

In [7]:
## Train FastText model using Skip-gram
model = fasttext.train_unsupervised("fasttext_train.txt", model='skipgram', dim=50, epoch=20, minCount=1, ws=5)

## Save the model
model.save_model("custom_fasttext_skipgram.bin")

In [8]:
## Example: Get the vector for the word 'love' 
love_vector = model.get_word_vector("ရခိုင်")
print("Vector for 'ရခိုင်':", love_vector) 

Vector for 'ရခိုင်': [-1.0041574e-03  1.0815905e-03 -2.5785216e-03  9.7449774e-05
  5.6208349e-03 -4.7307415e-03 -2.2986415e-03 -1.0629549e-03
  2.0623573e-03  1.4940150e-03  9.5819577e-04  7.5829669e-04
  4.4705607e-03  6.9303397e-04  1.9693540e-03 -6.0366205e-04
 -1.5917385e-03 -3.5784587e-03  1.0524314e-03 -1.1997729e-03
 -1.2166420e-03 -1.4223205e-03  3.7401970e-03  4.7307028e-03
  9.6115982e-04  2.7013619e-03  4.4234945e-03  1.6331029e-03
 -9.6729322e-04 -1.6283244e-03 -4.9724081e-03 -2.7744211e-03
  1.0396333e-03 -1.1112135e-03 -1.4830615e-03  1.0704593e-03
  8.2971802e-04  1.1066482e-03 -4.2416691e-03 -1.9462983e-03
 -2.4169362e-03 -3.0321453e-03  2.0079389e-03  2.9728371e-03
 -1.0169896e-03  1.9076661e-03 -4.4356915e-04  3.0868954e-03
 -2.9508965e-04 -8.6111145e-04]


In [9]:
## Inference: Load the custom FastText model
model = fasttext.load_model("custom_fasttext_skipgram.bin")

## Convert a sentence into a vector by averaging word vectors
def sentence_to_vector(sentence, model, vector_size=50):
    cleaned_sentence = preprocess_text(sentence)
    tokens = cleaned_sentence.split()
    vectors = [model.get_word_vector(word) for word in tokens]
    
    ## If no word in the sentence is in the model, return a zero vector
    if len(vectors) == 0:
        return np.zeros(vector_size)
    
    ## Average of all word vectors
    return np.mean(vectors, axis=0)

## Convert the entire dataset into sentence vectors
features = np.array([sentence_to_vector(sentence, model, vector_size=50) for sentence in dataset])

## Example: Check the sentence vector for the first sentence 
print(f"Sentence vector of '{dataset[0]}' is \n", features[0]) 

Sentence vector of 'မြန်မာ ပြည် ရှေ့ရေး အကြောင်း ဆွေးနွေးသွားတာ အရမ်း ကောင်းတယ်။' is 
 [ 7.3532260e-04 -1.4032103e-03 -1.4431754e-04  9.0734847e-04
  4.3779591e-04  1.3493855e-03 -1.2273055e-03  1.6483615e-04
  5.1657954e-04  5.4288591e-04 -1.0835655e-04  1.1772254e-03
 -2.2758474e-04  7.2686194e-04 -7.2697445e-04  2.1426387e-04
 -4.0157750e-04  1.0967721e-03 -5.3939450e-04 -4.8893003e-04
 -8.5119630e-04  4.0826169e-04 -3.5447814e-04  3.4636338e-04
  1.2930875e-03 -7.3039631e-04  5.9102796e-04  1.1557258e-03
  1.0218156e-03  1.3704652e-03 -4.7481419e-05 -8.0336496e-04
  2.2384613e-04  5.5536436e-04  9.0422377e-04  1.3629878e-03
  6.8195083e-04 -9.6104741e-05  5.5928726e-04  6.7523919e-04
  2.4536284e-04  2.0215087e-04 -2.3507029e-04  7.3567506e-05
  7.7349963e-05 -2.3214097e-04 -3.5663167e-04 -5.3712743e-04
  2.5129699e-04 -1.3299227e-03]


In [10]:
## Classifier
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

## Test cosine similarity between two sentences
similarity = cosine_similarity(features[1], features[2])
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.49795577


In [12]:
## Inference
test = "မင်းဆိုး မင်းညစ် အယုတ်တမာ ကျဆုံးမှာပါ။"
test_vector = sentence_to_vector(test, model)
preds = [cosine_similarity(source, test_vector) for source in features]
pred = encoded_lbls[np.argmax(preds)]
predicted_emotion = [emotion for emotion, idx in target_mapping.items() if idx == pred]
print("Prediction :", predicted_emotion[0])

Prediction : Negative
